In [1]:
import numpy as np
import torch
%matplotlib inline
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

random_seed = 1234
np.random.seed(random_seed)

In [2]:
N_train = 10000
N_bound = 500

train_data = lhs(10,N_train)

In [ ]:
bound_x_0_list = []
for i in range(10):
    bound_x_temp = lhs(10, N_bound)
    bound_x_temp[:, i:i+1] = np.zeros((N_bound, 1))
    bound_x_0_list.append(bound_x_temp)

bound_x_0 = np.concatenate(bound_x_0_list, axis=0)

bound_x_1_list = []
for i in range(10):
    bound_x_temp = lhs(10, N_bound)
    bound_x_temp[:, i:i+1] = np.ones((N_bound, 1))
    bound_x_1_list.append(bound_x_temp)

bound_x_1 = np.concatenate(bound_x_1_list, axis=0)

bound_x_data = np.concatenate((bound_x_0, bound_x_1), axis=0)

bound_x_data = numpy_to_tensor(bound_x_data, var_name="bound_x_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

## Exact u

In [4]:
def u_solution(data_input):
    
    x1 = data_input[:, 0:1]
    x2 = data_input[:, 1:2]
    x3 = data_input[:, 2:3]
    x4 = data_input[:, 3:4]
    x5 = data_input[:, 4:5]
    x6 = data_input[:, 5:6]
    x7 = data_input[:, 6:7]
    x8 = data_input[:, 7:8]
    x9 = data_input[:, 8:9]
    x10 = data_input[:, 9:10]
    
    u_value = x1**2-x2**2+x3**2-x4**2+x5*x6+x7*x8*x9*x10
    
    return u_value

### Test Data

In [ ]:
np.random.seed(5678)
test_data = lhs(10,N_train)
test_u = u_solution(test_data)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

## Loss_f

In [6]:
def get_loss_f(train_data_input, PINNs, return_sequence='not'):
    
    # Extract f_data into individual variables (x1, x2, ..., x10)
    x1_inside = train_data_input[:, 0:1]
    x2_inside = train_data_input[:, 1:2]
    x3_inside = train_data_input[:, 2:3]
    x4_inside = train_data_input[:, 3:4]
    x5_inside = train_data_input[:, 4:5]
    x6_inside = train_data_input[:, 5:6]
    x7_inside = train_data_input[:, 6:7]
    x8_inside = train_data_input[:, 7:8]
    x9_inside = train_data_input[:, 8:9]
    x10_inside = train_data_input[:, 9:10]

    x1_inside = numpy_to_tensor(x1_inside, var_name="x1_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x2_inside = numpy_to_tensor(x2_inside, var_name="x2_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x3_inside = numpy_to_tensor(x3_inside, var_name="x3_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x4_inside = numpy_to_tensor(x4_inside, var_name="x4_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x5_inside = numpy_to_tensor(x5_inside, var_name="x5_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x6_inside = numpy_to_tensor(x6_inside, var_name="x6_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x7_inside = numpy_to_tensor(x7_inside, var_name="x7_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x8_inside = numpy_to_tensor(x8_inside, var_name="x8_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x9_inside = numpy_to_tensor(x9_inside, var_name="x9_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True)
    x10_inside = numpy_to_tensor(x10_inside, var_name="x10_inside", value_range_dim = False, to_torch = True, to_cuda = True, requires_grad = True) 
    
    E_inside = PINNs1(torch.cat((x1_inside,x2_inside,x3_inside,x4_inside,x5_inside,x6_inside,x7_inside,x8_inside,x9_inside,x10_inside),1))
    
    E_xx1 = compute_higher_order_derivatives(E_inside, [x1_inside,x1_inside])
    E_xx2 = compute_higher_order_derivatives(E_inside, [x2_inside,x2_inside])
    E_xx3 = compute_higher_order_derivatives(E_inside, [x3_inside,x3_inside])
    E_xx4 = compute_higher_order_derivatives(E_inside, [x4_inside,x4_inside])
    E_xx5 = compute_higher_order_derivatives(E_inside, [x5_inside,x5_inside])
    E_xx6 = compute_higher_order_derivatives(E_inside, [x6_inside,x6_inside])
    E_xx7 = compute_higher_order_derivatives(E_inside, [x7_inside,x7_inside])
    E_xx8 = compute_higher_order_derivatives(E_inside, [x8_inside,x8_inside])
    E_xx9 = compute_higher_order_derivatives(E_inside, [x9_inside,x9_inside])
    E_xx10 = compute_higher_order_derivatives(E_inside, [x10_inside,x10_inside])
    
    loss_term = E_xx1+E_xx2+E_xx3+E_xx4+E_xx5+E_xx6+E_xx7+E_xx8+E_xx9+E_xx10
    
    if return_sequence == 'yes':
        return torch.square(loss_term)
    else:
        return torch.mean(torch.square(loss_term))

# PINN

In [7]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(random_seed)

In [8]:
net_settings_for_PINNs1 = NetSetting(input_dims=10, hidden_neurons_list=[100]*5, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam([{'params': PINNs1.parameters()}], lr=1e-3)

In [9]:
def create_mini_batches(train_data_new, batch_size):
    
    indices = np.arange(len(train_data_new))
    np.random.shuffle(indices)
    
    for start_idx in range(0, len(train_data_new) - batch_size + 1, batch_size):
        
        excerpt_batch = indices[start_idx:start_idx + batch_size]
        train_data_batch = train_data_new[excerpt_batch,:]

        yield train_data_batch, excerpt_batch

In [10]:
def update_collocation_points(Number_select, Number_all):

    # Training data
    train_data = lhs(10,N_train)
    
    random_indices_perm = torch.randperm(Number_all)
    random_indices = random_indices_perm[:Number_select]
    
    train_data_new = train_data[random_indices,:]

    return train_data_new

In [11]:
loss_f_1 = []
loss_b_1 = []
loss_all_1 = []
test_loss_1 = []

In [ ]:
nIter2 = 1000
it = 0
Number_select = 10000
Number_all = 10000
batch_size = 1000

while it<nIter2 :
    
    if it:
        train_data = update_collocation_points(Number_select, Number_all)
    
    for train_data_batch, batch_indices in create_mini_batches(train_data, batch_size):
        
        ##### loss bound ######
        E_bound = PINNs1(bound_x_data)
        real_bound = u_solution(bound_x_data)
        loss_bound = torch.mean(torch.square(E_bound-real_bound))

        #### loss f  ######     
        loss_f = get_loss_f(train_data_batch, PINNs1, return_sequence='not')

        #####loss PI#######
        loss = loss_bound+loss_f 
  
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
        
    #########  test_loss NRMSE  #########
    pre_u = PINNs1(test_data)  
    test_loss = relative_l2_torch(pre_u,test_u)
    test_loss_1.append(test_loss)
    #########  test_loss NRMSE  #########

    ##############Record###############
    loss_all_1.append(loss.item())
    loss_f_1.append(loss_f.item())
    loss_b_1.append(loss_bound.item())
    ##############Record############### 
    
    if it % 100 == 0:
        print(f'It: {it}, train_loss: {loss.item()}, test_loss: {test_loss}, best_test_loss: {min(test_loss_1):.4f}')
    it = it + 1            